In [1]:
!pip install sklearn_crfsuite

In [2]:
import numpy as np
import pandas as pd

from itertools import chain

import nltk
import sklearn
import scipy.stats
from sklearn.metrics import make_scorer
from sklearn.model_selection import RandomizedSearchCV

import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics

import re

In [3]:
train_file = open("/content/hi-ud-train.conllu", "r", encoding="utf-8")
test_file = open("/content/hi-ud-test .conllu", "r", encoding="utf-8")

In [4]:
train_rows = []
for x in train_file:
  train_rows.append(str(x))

test_rows = []
for x in test_file:
  test_rows.append(str(x))

#print(train_rows[0])
#print(test_rows[0])

In [5]:
train_sents = []
sent = []
for x in train_rows[1:]:
  if x.split(',')[0] is '':
    train_sents.append(sent)
    sent = []
  elif x is train_rows[-1]:
    train_sents.append(sent)
  else:
    word = x.split(',')[1]
    tag = x.split(',')[2][:-1]
    sent.append((word,tag))

In [6]:
test_sents = []
sent = []
for x in test_rows[1:]:
  if x.split('\t')[0] is '':
    test_sents.append(sent)
    sent = []
  elif x is test_rows[-1]:
    test_sents.append(sent)
  else:
    #print(x)
    word = x.split('\t')[1]
    tag = x.split('\t')[2][:-1]
    sent.append((word,tag))

In [7]:
print(train_sents[0])

[('yaha', 'DET'), ('eSiyA', 'PROPN'), ('kI', 'ADP'), ('sabase', 'ADV'), ('badZI', 'ADJ'), ('masjixoM', 'NOUN'), ('meM', 'ADP'), ('se', 'ADP'), ('eka', 'NUM'), ('hE', 'AUX'), ('.', 'PUNCT')]


In [8]:
print(test_sents[0])

[('rAmAyaNa', 'PROPN'), ('kAla', 'PROPN'), ('meM', 'ADP'), ('BagavAna', 'NOUN'), ('rAma', 'PROPN'), ('ke', 'ADP'), ('puwra', 'NOUN'), ('kuSa', 'PROPN'), ('kI', 'ADP'), ('rAjaXAnI', 'NOUN'), ('kuSAvawI', 'PROPN'), ('ko', 'ADP'), ('483', 'PROPN'), ('IsA', 'PROPN'), ('pUrva', 'PROPN'), ('buxXa', 'PROPN'), ('ne', 'ADP'), ('apane', 'PRON'), ('aMwima', 'ADJ'), ('viSrAma', 'NOUN'), ('ke', 'ADP'), ('lie', 'ADP'), ('cunA', 'VERB'), ('.', 'PUNCT')]


In [9]:
def word2features(sent, i):
    word = sent[i][0]
    
    features = {
        'word.lower()': word.lower(),
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'is_first_capital':int(word[:1].isupper()),
        'is_first_word': int(i==0),
        'is_last_word':int(i==len(sent)-1),
        'is_alphanumeric': int(bool((re.match('^(?=.*[0-9]$)(?=.*[a-zA-Z])',word)))),
        'prefix_1':word[:1],
        'prefix_2':word[:2],
        'prefix_3':word[:3],
        'prefix_4':word[:4],
        'suffix_1':word[-1:],
        'suffix_2':word[-2:],
        'suffix_3':word[-3:],
        'suffix_4':word[-4:],
        'has_hyphen': 1 if '-' in word else 0 
    }
    if i > 0:
        word1 = sent[i-1][0]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
        })
    else:
        features['BOS'] = True

    if i < len(sent)-1:
        word1 = sent[i+1][0]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
        })
    else:
        features['EOS'] = True

    return features


def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2tags(sent):
    return [postag for token, postag in sent]

Feature description:-


*   'word.lower()' :- Lowercase version of the word
*   'word.isupper()' :- Whether the word is in uppercase or not
*   'word.istitle()' :- Whether the word is digit or not
*   'is_first_capital' :- Whether first letter of the word is in uppercase or not
*   'is_first_word' :- Whether the word is first word of the sentence or not
*   'is_last_word' :- Whether the word is last word of the sentence or not
*   'is_alphanumeric' :- Whether the word is alphanumeric or not
*   'prefix_i' :- perfix of length i 
*   'suffix_i' :- suffix of length i
*   'has_hyphen' :- Whether the word has a hyphen or not
*   '-1:word.lower()' :- Lowercase version of the previous word
*   '-1:word.istitle()' :- Whether the previous word is title or not
*   '-1:word.isupper()' :- Whether the previous word is in uppercase or not
*   '+1:word.lower()' :- Lowercase version of the next word
*   '+1:word.istitle()' :- Whether the previous word is title or not
*   '+1:word.isupper()' :- Whether the previous word is in uppercase or not
















In [10]:
sent2features(train_sents[0])[0]

{'+1:word.istitle()': False,
 '+1:word.isupper()': False,
 '+1:word.lower()': 'esiya',
 'BOS': True,
 'has_hyphen': 0,
 'is_alphanumeric': 0,
 'is_first_capital': 0,
 'is_first_word': 1,
 'is_last_word': 0,
 'prefix_1': 'y',
 'prefix_2': 'ya',
 'prefix_3': 'yah',
 'prefix_4': 'yaha',
 'suffix_1': 'a',
 'suffix_2': 'ha',
 'suffix_3': 'aha',
 'suffix_4': 'yaha',
 'word.isdigit()': False,
 'word.istitle()': False,
 'word.isupper()': False,
 'word.lower()': 'yaha'}

In [11]:
X_train = [sent2features(s) for s in train_sents]
y_train = [sent2tags(s) for s in train_sents]

X_test = [sent2features(s) for s in test_sents]
y_test = [sent2tags(s) for s in test_sents]

In [12]:
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=True
)
crf.fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/base.py:197: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


CRF(algorithm='lbfgs', all_possible_states=None, all_possible_transitions=True,
    averaging=None, c=None, c1=0.1, c2=0.1, calibration_candidates=None,
    calibration_eta=None, calibration_max_trials=None, calibration_rate=None,
    calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
    gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=100,
    max_linesearch=None, min_freq=None, model_filename=None, num_memories=None,
    pa_type=None, period=None, trainer_cls=None, variance=None, verbose=False)

In [13]:
y_pred_train = crf.predict(X_train)
y_pred_test = crf.predict(X_test)
train_accuracy = sklearn_crfsuite.metrics.flat_accuracy_score(y_train, y_pred_train)
test_accuracy = sklearn_crfsuite.metrics.flat_accuracy_score(y_test, y_pred_test)

In [14]:
print(train_accuracy)
print(test_accuracy)

0.9969776609724047
0.8531228551818806


In [15]:
tags = []
for y in y_train:
  for x in y:
    if x not in tags:
      tags.append(x)

for y in y_test:
  for x in y:
    if x not in tags:
      tags.append(x)

print(tags)

['DET', 'PROPN', 'ADP', 'ADV', 'ADJ', 'NOUN', 'NUM', 'AUX', 'PUNCT', 'PRON', 'VERB', 'CCONJ', 'PART', 'COMMA', 'SCONJ', 'X']


In [18]:
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    max_iterations=100,
    all_possible_transitions=True
)
params_space = {
    'c1': scipy.stats.expon(scale=0.5),
    'c2': scipy.stats.expon(scale=0.05),
}

f1_scorer = make_scorer(metrics.flat_accuracy_score)

rs = RandomizedSearchCV(crf, params_space,
                        cv=3,
                        verbose=1,
                        n_jobs=-1,
                        n_iter=50,
                        scoring=f1_scorer)
rs.fit(X_train, y_train)

Fitting 3 folds for each of 50 candidates, totalling 150 fits


/usr/local/lib/python3.6/dist-packages/sklearn/base.py:197: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 150 out of 150 | elapsed:  4.6min finished


RandomizedSearchCV(cv=3, error_score=nan,
                   estimator=CRF(algorithm='lbfgs', all_possible_states=None,
                                 all_possible_transitions=True, averaging=None,
                                 c=None, c1=None, c2=None,
                                 calibration_candidates=None,
                                 calibration_eta=None,
                                 calibration_max_trials=None,
                                 calibration_rate=None,
                                 calibration_samples=None, delta=None,
                                 epsilon=None, error_sensitive=None, gamma=None,
                                 keep_...
                                 trainer_cls=None, variance=None,
                                 verbose=False),
                   iid='deprecated', n_iter=50, n_jobs=-1,
                   param_distributions={'c1': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f4f1edf2128>,
                    

In [19]:
print('best params:', rs.best_params_)
print('best CV score:', rs.best_score_)
print('model size: {:0.2f}M'.format(rs.best_estimator_.size_ / 1000000))

best params: {'c1': 0.011189679492593299, 'c2': 0.07756578581396453}
best CV score: 0.8590887329284292
model size: 0.81M


In [20]:
crf = rs.best_estimator_

In [21]:
y_pred_train = crf.predict(X_train)
y_pred_test = crf.predict(X_test)
train_accuracy = sklearn_crfsuite.metrics.flat_accuracy_score(y_train, y_pred_train)
test_accuracy = sklearn_crfsuite.metrics.flat_accuracy_score(y_test, y_pred_test)

In [22]:
print(train_accuracy)
print(test_accuracy)

0.9988173455978975
0.849691146190803


In [23]:
sorted_labels = sorted(
    tags,
    key=lambda name: (name[1:], name[0])
)

In [29]:
print("For train dataset:-\n")
print(metrics.flat_classification_report(
    y_train, y_pred_train, labels=sorted_labels, digits=3
))

For train dataset:-

              precision    recall  f1-score   support

           X      1.000     1.000     1.000         2
        PART      1.000     1.000     1.000       163
       CCONJ      1.000     1.000     1.000       150
       SCONJ      1.000     1.000     1.000        61
         ADJ      1.000     1.000     1.000       570
         ADP      0.999     1.000     1.000      1387
         ADV      1.000     0.991     0.995       111
        VERB      1.000     0.991     0.995       640
         DET      0.996     1.000     0.998       231
       COMMA      1.000     1.000     1.000       114
        NOUN      0.999     1.000     1.000      1597
        PRON      1.000     0.998     0.999       431
       PROPN      1.000     0.999     0.999       708
         NUM      1.000     1.000     1.000       152
       PUNCT      1.000     1.000     1.000       563
         AUX      0.992     1.000     0.996       730

    accuracy                          0.999      7610
   ma

' '

In [25]:
print("For test dataset:-\n")
print(metrics.flat_classification_report(
    y_test, y_pred_test, labels=sorted_labels, digits=3, zero_division=0
))

For test dataset:-

              precision    recall  f1-score   support

           X      0.000     0.000     0.000         0
        PART      1.000     1.000     1.000        33
       CCONJ      1.000     1.000     1.000        25
       SCONJ      0.600     1.000     0.750         3
         ADJ      0.667     0.723     0.694        94
         ADP      0.944     0.974     0.959       309
         ADV      0.667     0.476     0.556        21
        VERB      0.867     0.859     0.863        99
         DET      0.805     0.917     0.857        36
       COMMA      0.000     0.000     0.000         0
        NOUN      0.787     0.875     0.829       329
        PRON      0.844     0.831     0.837        65
       PROPN      0.649     0.497     0.562       145
         NUM      0.917     0.880     0.898        25
       PUNCT      1.000     0.836     0.911       134
         AUX      0.930     0.950     0.940       139

   micro avg      0.850     0.850     0.850      1457
   mac

In [26]:
from collections import Counter

def print_transitions(trans_features):
    for (label_from, label_to), weight in trans_features:
        print("%-6s -> %-7s %0.6f" % (label_from, label_to, weight))

print("Top 10 most common transition features:")
print_transitions(Counter(crf.transition_features_).most_common(10))

print("\nTop 10 least common transition features:")
print_transitions(Counter(crf.transition_features_).most_common()[-10:])

Top 10 most common transition features:
VERB   -> AUX     4.144519
PROPN  -> PROPN   3.433338
ADJ    -> NOUN    2.925595
NUM    -> NOUN    2.389616
NOUN   -> ADP     2.158102
PROPN  -> ADP     2.110155
DET    -> NOUN    2.078512
AUX    -> AUX     2.053971
PRON   -> ADP     1.864570
VERB   -> SCONJ   1.671803

Top 10 least common transition features:
ADV    -> ADV     -1.321835
PROPN  -> DET     -1.372187
ADV    -> AUX     -1.385762
DET    -> CCONJ   -1.508132
ADP    -> COMMA   -1.515826
PROPN  -> AUX     -1.531533
ADJ    -> PRON    -1.818604
CCONJ  -> AUX     -1.834209
ADJ    -> ADP     -2.219360
DET    -> ADP     -2.848158
